In [1]:
!pip install fastai==1.0.52

    100% |████████████████████████████████| 225kB 7.8MB/s ta 0:00:011
  Found existing installation: fastai 1.0.51
    Uninstalling fastai-1.0.51:
      Successfully uninstalled fastai-1.0.51
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.tsv', 'test.tsv', 'sampleSubmission.csv']


In [3]:
!mkdir stuff

In [4]:
import fastai
from fastai import * 
from fastai.text import *
import random
import torch

In [5]:
'''Setting seed for reproducibility'''
SEED = 2019

# python RNG
random.seed(SEED)

# pytorch RNGs
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)

# numpy RNG
np.random.seed(SEED)

In [6]:
train_df = pd.read_table('../input/train.tsv')
test_df = pd.read_table('../input/test.tsv')

In [7]:
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [8]:
train_df.isnull().sum()

PhraseId      0
SentenceId    0
Phrase        0
Sentiment     0
dtype: int64

In [9]:
train_df['Sentiment'].value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [10]:
train_df = pd.DataFrame({'label':train_df.Sentiment,
                        'text':train_df.Phrase})

In [11]:
test_df = pd.DataFrame({'label':test_df.PhraseId,
    'text':test_df.Phrase})

In [12]:
lm_df = [train_df,test_df]
lm_df = pd.concat(lm_df)
lm_df.to_csv('./stuff/lm.csv',index=False)

In [13]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(train_df, stratify = train_df['label'], test_size = 0.1, random_state = SEED)

In [14]:
df_trn.shape, df_val.shape

((140454, 2), (15606, 2))

In [15]:
# Language model data
data_lm = TextLMDataBunch.from_csv("",'./stuff/lm.csv',bs=128, min_freq=1)

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, min_freq=1, vocab=data_lm.train_ds.vocab, bs=128)

In [ ]:
# data_lm.show_batch()

In [16]:
learn = language_model_learner(data_lm,arch=AWD_LSTM, drop_mult=0.3)

In [ ]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [17]:
lr = 7e-2
learn.fit_one_cycle(1, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.244061,3.736418,0.354958,00:44


In [18]:
learn.unfreeze()

In [ ]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [19]:
learn.fit_one_cycle(10, slice(2e-4,2e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.669571,3.599040,0.371476,00:51
1,3.446634,3.325278,0.410122,00:51
2,3.159054,3.010561,0.460570,00:51
3,2.877957,2.759402,0.503963,00:51
4,2.670769,2.585641,0.535300,00:51
5,2.531279,2.470985,0.555799,00:51
6,2.405631,2.399258,0.569727,00:51
7,2.352085,2.361009,0.575982,00:51
8,2.305193,2.345312,0.579035,00:51
9,2.306712,2.343266,0.579303,00:51


In [ ]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [20]:
learn.fit_one_cycle(10, slice(5e-5,5e-4), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,2.288424,2.339746,0.580317,00:51
1,2.287187,2.325426,0.582671,00:51
2,2.288495,2.301875,0.586654,00:51
3,2.253847,2.279914,0.592049,00:51
4,2.224537,2.257122,0.595149,00:51
5,2.189158,2.239897,0.598408,00:51
6,2.190601,2.229133,0.600501,00:51
7,2.169977,2.222655,0.602083,00:51
8,2.158298,2.219888,0.602597,00:51
9,2.162124,2.219536,0.602562,00:51


In [ ]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [21]:
learn.fit_one_cycle(10, slice(5e-6,5e-5), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,2.156412,2.219213,0.602661,00:51
1,2.161321,2.218105,0.602800,00:51
2,2.150564,2.216268,0.603247,00:51
3,2.147057,2.214275,0.603581,00:51
4,2.149361,2.212205,0.603864,00:51
5,2.143698,2.210717,0.604246,00:51
6,2.138908,2.209741,0.604492,00:51
7,2.162535,2.208885,0.604660,00:51
8,2.151944,2.208887,0.604700,00:51
9,2.135075,2.208623,0.604715,00:51


In [ ]:
# learn.recorder.plot_lr(show_moms=True)

In [ ]:
# learn.recorder.plot_metrics()

In [22]:
learn.save_encoder('ft_enc')

In [23]:
learn = text_classifier_learner(data_clas,arch=AWD_LSTM,drop_mult=0.3)

In [24]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(df_trn['label']),
                                                 df_trn['label'])

In [25]:
learn.loss_func.func = torch.nn.CrossEntropyLoss(weight=torch.from_numpy(class_weights).float().cuda())

In [26]:
kappa = KappaScore()

In [27]:
learn.metrics = [kappa, accuracy]

In [28]:
learn.load_encoder('ft_enc')
learn.freeze()

In [29]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [30]:
lr = 1e-1
learn.fit_one_cycle(1,lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,kappa_score,accuracy,time
0,1.364633,1.250474,0.266825,0.467256,00:24


In [31]:
learn.freeze_to(-2)

In [ ]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [32]:
learn.fit_one_cycle(20,max_lr=slice(1e-2/2.6**4,1e-2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,kappa_score,accuracy,time
0,1.286536,1.180102,0.312718,0.515315,00:26
1,1.231110,1.114737,0.340646,0.527425,00:28
2,1.184089,1.068013,0.348104,0.541266,00:25
3,1.086418,1.029168,0.369793,0.558311,00:28
4,1.055533,1.002844,0.368324,0.550878,00:27
5,1.002971,0.970636,0.370946,0.541010,00:28
6,0.957631,0.965612,0.412330,0.589965,00:27
7,0.923272,0.948138,0.406290,0.572408,00:28
8,0.885305,0.987123,0.420780,0.588684,00:27
9,0.836943,0.961506,0.402666,0.570486,00:27


In [ ]:
learn.freeze_to(-3)

In [ ]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [33]:
learn.fit_one_cycle(19,max_lr=slice(1e-4/2.6**4,1e-4), moms=(0.8,0.7))

epoch,train_loss,valid_loss,kappa_score,accuracy,time
0,0.603965,1.138436,0.432151,0.593426,00:28
1,0.593409,1.146091,0.436641,0.599193,00:26
2,0.589857,1.125130,0.431920,0.593682,00:26
3,0.594826,1.162328,0.433809,0.595476,00:26
4,0.589464,1.156544,0.437465,0.599577,00:28
5,0.612219,1.170068,0.435877,0.597655,00:28
6,0.597981,1.164328,0.437635,0.599769,00:28
7,0.588599,1.127834,0.439022,0.601115,00:26
8,0.596233,1.185240,0.442156,0.604511,00:28
9,0.585142,1.218532,0.433566,0.595604,00:28


In [ ]:
# learn.unfreeze()

In [ ]:
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [ ]:
# learn.fit_one_cycle(3,max_lr=slice(1e-5/2.6**4,1e-5), moms=(0.8,0.7))

In [ ]:
# learn.save('final-model')

In [ ]:
# interp = ClassificationInterpretation.from_learner(learn)
# losses,idxs = interp.top_losses()

In [ ]:
# interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
# interp.most_confused(min_val=2)

In [ ]:
learn.export()

In [ ]:
test_loader = TextList.from_df(df=test_df,cols='text')

In [ ]:
learn = load_learner(path='',file='export.pkl',test=test_loader,bs=128)

In [ ]:
probs,_ = learn.get_preds(ds_type=DatasetType.Test,ordered=True)

In [ ]:
submission_df = pd.read_csv('../input/sampleSubmission.csv')

In [ ]:
submission_df.head()

In [ ]:
submission_df['Sentiment'] = probs.argmax(1)

In [ ]:
submission_df.to_csv('results.csv',index=False)